In [1]:
# import all libraries
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [2]:
# all used tickers
tickers = ['LLY', 'NVO', 'JNJ', 'MRK', 'ABBV', 'MRNA',
           'NVS', 'AZN', 'PFE', 'AMGN', 'PPH', 'IHE', 'PJP']

dfs = {}

for symbol in tickers:
    # for each symbol, load the pkl file and store it in the dictionary as a df
    dfs[f"{symbol}_df"] = pd.read_csv(f"pharma-data/clean-data/stocks/{symbol}_df.csv", index_col='Date', parse_dates=True)

In [5]:
# read in overdose data
overdose_df = pd.read_csv("pharma-data/clean-data/overdose_deaths.csv", index_col='Date', parse_dates=True)

In [6]:
# print the first 2 rows of each df
for df_name, df in dfs.items():
    print(df_name + "\n")
    print(df.head(2))
    print("")

LLY_df

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2020-01-06  131.419998  132.559998  130.940002  132.259995  124.553093   
2020-01-07  131.699997  132.929993  131.699997  132.509995  124.788521   

             Volume  
Date                 
2020-01-06  2102900  
2020-01-07  2448300  

NVO_df

                 Open   High        Low      Close  Adj Close   Volume
Date                                                                  
2020-01-06  28.495001  28.59  28.434999  28.504999  26.977007  2495000
2020-01-07  28.500000  28.50  28.280001  28.475000  26.948616  2080600

JNJ_df

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2020-01-06  144.000000  144.199997  142.850006  144.100006  128.432449   
2020-01-07  144.009995  145.449997  141.380005  144.979996  129.216751   

        

In [7]:
# sanity check
overdose_df.head()

,US Overdose Deaths
Date,
2012-01-01,1
2013-01-01,1
2015-01-01,4
2016-01-01,4
2017-01-01,1


In [8]:
# prep dict for the merged data
merged_dfs = {}

for df_name, df in dfs.items():
    # convert index to datetime
    df.index = pd.to_datetime(df.index)

    # merge data frames based on date index, using inner join to keep only rows with matching dates. (keeps trading days)
    merged_dfs[df_name] = pd.merge(df, overdose_df, how='inner', left_index=True, right_index=True)

In [9]:
# send merged dfs to csv in /merged-dfs

directory = 'pharma-data/merged-dfs'

for df_name, df in merged_dfs.items():
    df.to_csv(f"{directory}/{df_name}-overdose.csv")

In [10]:
# check merge worked
for df_name, df in merged_dfs.items():
    print(df_name + "\n")
    print(df.head())
    print("")

LLY_df

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2020-01-06  131.419998  132.559998  130.940002  132.259995  124.553093   
2020-01-07  131.699997  132.929993  131.699997  132.509995  124.788521   
2020-01-08  132.460007  134.210007  132.009995  133.710007  125.918610   
2020-01-09  134.550003  136.360001  134.009995  135.919998  127.999825   
2020-01-10  135.779999  138.270004  135.529999  138.000000  129.958572   

             Volume  US Overdose Deaths  
Date                                     
2020-01-06  2102900                   3  
2020-01-07  2448300                   4  
2020-01-08  5188600                   3  
2020-01-09  4522800                   4  
2020-01-10  4177600                   3  

NVO_df

                 Open       High        Low      Close  Adj Close   Volume  \
Date                                                                         
2020-01-06  28

In [11]:
# check for correlation among columns. 
for df_name, df in merged_dfs.items():
    print(df_name + "\n")
    print(df.corr())
    print("")

LLY_df

                        Open      High       Low     Close  Adj Close  \
Open                1.000000  0.999190  0.999399  0.998643   0.998621   
High                0.999190  1.000000  0.999206  0.999486   0.999455   
Low                 0.999399  0.999206  1.000000  0.999336   0.999290   
Close               0.998643  0.999486  0.999336  1.000000   0.999938   
Adj Close           0.998621  0.999455  0.999290  0.999938   1.000000   
Volume             -0.280434 -0.264805 -0.287664 -0.273253  -0.272801   
US Overdose Deaths  0.048134  0.049043  0.047823  0.048744   0.048032   

                      Volume  US Overdose Deaths  
Open               -0.280434            0.048134  
High               -0.264805            0.049043  
Low                -0.287664            0.047823  
Close              -0.273253            0.048744  
Adj Close          -0.272801            0.048032  
Volume              1.000000            0.044311  
US Overdose Deaths  0.044311            1.000000  

In [13]:
# Find the pearson correlation coefficient among closing price & deaths for the day
for df_name, df in merged_dfs.items():
    p_coef, p_val = stats.pearsonr(df['Close'], df['US Overdose Deaths'])
    print(f"{df_name} Pearson Coef: {p_coef}" + "\n")

LLY_df Pearson Coef: 0.04874373570493411

NVO_df Pearson Coef: 0.04213463414174276

JNJ_df Pearson Coef: 0.07321569285564153

MRK_df Pearson Coef: -0.05485610618088754

ABBV_df Pearson Coef: 0.027967184918957513

MRNA_df Pearson Coef: 0.11939285838155195

NVS_df Pearson Coef: -0.011327466803019222

AZN_df Pearson Coef: 0.01486254650829061

PFE_df Pearson Coef: 0.018143353574079122

AMGN_df Pearson Coef: 0.011967264593087105

PPH_df Pearson Coef: 0.051777544978080375

IHE_df Pearson Coef: 0.0739239222743337

PJP_df Pearson Coef: 0.10050384599742675



In [16]:
# check for the best fit - linear, quadratic, or cubic between close and US covid deaths
pearson_corr = {}
for df_name, df in merged_dfs.items():
    pearson_corr[df_name] = df['Close'].corr(df['US Overdose Deaths'])

# fit models and calculate r-squared
r_squared_values = {}
for df_name, df in merged_dfs.items():
    x = df['Close'].values.reshape(-1, 1)
    y = df['US Overdose Deaths'].values

    # linear regression
    model_linear = LinearRegression()
    model_linear.fit(x, y)
    y_pred_linear = model_linear.predict(x)
    r_squared_values[f"{df_name}_linear"] = model_linear.score(x, y)

    # quadratic regression
    poly = PolynomialFeatures(degree=2)
    x_poly = poly.fit_transform(x)
    model_quad = LinearRegression()
    model_quad.fit(x_poly, y)
    y_pred_quad = model_quad.predict(x_poly)
    r_squared_values[f"{df_name}_quadratic"] = model_quad.score(x_poly, y)

    # cubic regression
    poly = PolynomialFeatures(degree=3)
    x_poly = poly.fit_transform(x)
    model_cubic = LinearRegression()
    model_cubic.fit(x_poly, y)
    y_pred_cubic = model_cubic.predict(x_poly)
    r_squared_values[f"{df_name}_cubic"] = model_cubic.score(x_poly, y)

# print r-squared values
print("\nR-squared values:")
count = 0
for model_name, r_squared in r_squared_values.items():
    print(f"{model_name}: {r_squared}")
    count+=1

    # after printing the 3 possibilities, print new line
    if count % 3 == 0:
        print("")


R-squared values:
LLY_df_linear: 0.002375951770472451
LLY_df_quadratic: 0.010651042707248148
LLY_df_cubic: 0.013573160623992164

NVO_df_linear: 0.0017753273942585635
NVO_df_quadratic: 0.013098936049379617
NVO_df_cubic: 0.013136031795593972

JNJ_df_linear: 0.005360537680331623
JNJ_df_quadratic: 0.0061194257564979315
JNJ_df_cubic: 0.0061694224258657115

MRK_df_linear: 0.0030091923853288183
MRK_df_quadratic: 0.0031706210346826857
MRK_df_cubic: 0.005249162224883919

ABBV_df_linear: 0.0007821634322913162
ABBV_df_quadratic: 0.007545658127585675
ABBV_df_cubic: 0.007549722344473819

MRNA_df_linear: 0.014254654632517183
MRNA_df_quadratic: 0.01432902954148818
MRNA_df_cubic: 0.01859699746960919

NVS_df_linear: 0.0001283115041734728
NVS_df_quadratic: 0.005092912932211813
NVS_df_cubic: 0.005219721371707786

AZN_df_linear: 0.00022089528871116482
AZN_df_quadratic: 0.005034758825948726
AZN_df_cubic: 0.005392382819194297

PFE_df_linear: 0.00032918127891423676
PFE_df_quadratic: 0.003240261265520905
PFE

In [17]:
# check example shape
merged_dfs['ABBV_df'].shape

(732, 7)